# Descripción del problema   

El código que teníamos no era muy claro, porque ha sido adaptar lo de ElasticityPINN a lo mio. Lo mejora será que haga desde cero la estructura que puedo necesitar para mi problema.   
En concreto, para resolver el problema de las PINNs, tenemos un entrenamiento de un NN con unos términos de la funcion coste, de modo que lo mejor, será tener un objeto entrenamiento que ponga todo junto, una NN, unos términos en la función coste, una optimización de los pesos. Una cosa que se ha barajado, es como voy a calcular la loss, si pesandola o como......   
Para hacerlo lo más modula posible, veamos que cosas querre tocar más a menudo. Por un dalo, la arquitectura o lo que sea que nos aproxima la función. Luego esta la función de pérdida y sus términos. Aqui es donde más cambios voy a realizar, y voy a usar multiples derivadas de los outputs respecto a los inputs.  

Después del cálculo de la loss, pues tenemos el proceso de actualización de los pesos, que es aplicar un optmizador a la derivada del grad loss, no parece mucho más.   

Y además los datos, que tiene que tener un procesado previo.  

Problema inverso para E, que ya funcionaba para la estimación, pero con los desplazamiento regulin.

In [3]:
#librerias

import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import re 
import json 
import time
import plotly.express as px
from dataclasses import dataclass
import deepxde as dde



Using backend: pytorch
Other supported backends: tensorflow.compat.v1, tensorflow, jax, paddle.
paddle supports more examples now and is recommended.


In [4]:
def keep_line(line):
    line_split=line.split()
    try:
        int(line_split[0])
        #si no dio error, okey entra
        return True
    except:
        return False
#vamos a hacer una función para cargar todos los datos de una   
@dataclass
class Data:
    folder: str
    load_stage: int
    base_dir:str = "/home/arturo/Documents/programacion_stuff/DeepElasticity/data/001-LinearElasticity"
    E: str = None
    initialPosition_data_pd: pd.DataFrame =None
    stress_data_pd: pd.DataFrame =None
    displacement_data_pd: pd.DataFrame=None
    restricted_data_pd: pd.DataFrame =None
    force_data_pd: pd.DataFrame =None 
    final_data_pd: pd.DataFrame =None
    collocation_data_np:pd.DataFrame =None
    Pos_min: np.ndarray = None
    Pos_max: np.ndarray = None

    @classmethod
    def keep_line(line):
        line_split=line.split()
        try:
            int(line_split[0])
            #si no dio error, okey entra
            return True
        except:
            return False

    def load_data(self,load_stage=10):
        # carga las posiciones de los nodos    
        initialPosition_data_path = f"{self.base_dir}/ARTURO_TEST_1/NODES.txt"

        with open(initialPosition_data_path,"r") as f:
            aux_list=f.readlines()

        initialPosition_data=[i for i in aux_list if keep_line(i)]
        initialPosition_data=[i.strip("\n") for i in initialPosition_data]
        initialPosition_data=[i.split() for i in initialPosition_data]
        initialPosition_data_np=np.array(initialPosition_data).astype(float)[:,[0,1,2,3]]
        self.initialPosition_data_pd=pd.DataFrame(initialPosition_data_np,columns=["Node","X","Y","Z"]).set_index("Node")
        self.Pos_min=self.initialPosition_data_pd.min().to_numpy()
        self.Pos_max=self.initialPosition_data_pd.max().to_numpy()
        #carga el streess
        if self.E is not None:
            stress_data_path = f"{self.base_dir}/{self.folder}/PSOL_{load_stage}_NODAL_STRESSES_E{self.E}.txt"
        else: 
            stress_data_path = f"{self.base_dir}/{self.folder}/PSOL_{load_stage}_NODAL_STRESSES.txt"
        with open(stress_data_path,"r") as f:
            aux_list=f.readlines()


        stress_data=[i for i in aux_list if keep_line(i)]
        stress_data=[i.strip("\n").strip() for i in stress_data]
        patron = "[-.\d]+E-*\d{3}|^\d{0,4}|0\.0000"

        stress_data=[re.findall(patron,i) for i in stress_data]
        stress_data=np.array(stress_data,dtype=float)
        self.stress_data_pd=pd.DataFrame(stress_data,columns=["Node","SX","SY","SZ","SXY","SYZ","SXZ"]).set_index("Node")

        # cargar datos de desplazamiento  
        if self.E is not None:
            displacement_data_path = f"{self.base_dir}/{self.folder}/PSOL_{load_stage}_NODAL_DISP_E{self.E}.txt"
        else:
            displacement_data_path = f"{self.base_dir}/{self.folder}/PSOL_{load_stage}_NODAL_DISP.txt"

        with open(displacement_data_path,"r") as f:
            aux_list=f.readlines()


        displacement_data=[i for i in aux_list if keep_line(i)]
        displacement_data=[i.strip("\n").strip() for i in displacement_data]
        patron = r"[-.\d]+E-*\d{3}|^\d{0,4}|-?\d+\.\d+|0\.0000"

        displacement_data=[re.findall(patron,i) for i in displacement_data]
        displacement_data=np.array(displacement_data,dtype=float)
        self.displacement_data_pd=pd.DataFrame(displacement_data,columns=["Node","UX","UY","UZ","USUM"]).set_index("Node")

        # cargar datos de boundaries en el movimiento
        #cargar los nodos fijos
        with open(f"{self.base_dir}/ARTURO_TEST_1/RESTRINGED_NODES.txt","r") as f:
            restricted_data=f.readlines()

        restricted_data=[i for i in restricted_data if keep_line(i)]
        restricted_data=[i.strip("\n") for i in restricted_data]
        restricted_data=[i.split() for i in restricted_data]
        restricted_data_np=np.array(restricted_data)[:,[0,1]]
        self.restricted_data_pd=pd.DataFrame(restricted_data_np,columns=["Node","Direccion"])
        self.restricted_data_pd=self.restricted_data_pd.groupby("Node")["Direccion"].apply(lambda x : list(x)).to_frame().sort_index()
        self.restricted_data_pd.index=self.restricted_data_pd.index.astype(int)
        self.restricted_data_pd=self.restricted_data_pd.sort_index()
        self.restricted_data_pd=self.restricted_data_pd.rename(columns={"Direccion":"Restricciones"})

        with open(f"{self.base_dir}/ARTURO_TEST_1/FORCE_ON_NODES.txt","r") as f:
            force_data=f.readlines()


        force_data=[i for i in force_data if keep_line(i)]
        force_data=[i.strip("\n") for i in force_data]
        force_data=[i.split() for i in force_data]
        force_data_np=np.array(force_data)[:,[0,1,2]]
        self.force_data_pd=pd.DataFrame(force_data_np,columns=["Node","Direccion_Fuerza","Fuerza"])
        self.force_data_pd["Fuerza"]=self.force_data_pd["Fuerza"].astype(float)
        self.force_data_pd=self.force_data_pd.set_index("Node")
        self.force_data_pd.index=self.force_data_pd.index.astype(int)


        #ponemos todos los datos en común usando los nodos como clave   
        self.final_data_pd=self.initialPosition_data_pd.merge(self.stress_data_pd,left_index=True,right_index=True,how="left").merge(self.force_data_pd,left_index=True,right_index=True,how="left").merge(self.displacement_data_pd,left_index=True,right_index=True,how="left").merge(self.restricted_data_pd,left_index=True,right_index=True,how="left")

        self.final_data_pd["Final_X"]=self.final_data_pd["X"]-self.final_data_pd["UX"]
        self.final_data_pd["Final_Y"]=self.final_data_pd["Y"]-self.final_data_pd["UY"]
        self.final_data_pd["Final_Z"]=self.final_data_pd["Z"]-self.final_data_pd["UZ"]

    def create_colloc_points(self,n_colloc,type="random_2"):

        if type=="random_1":

            X_coloc = np.random.uniform(self.Pos_min[0], self.Pos_max[0], (n_colloc, 1))
            Y_coloc = np.random.uniform(self.Pos_min[1], self.Pos_max[1], (n_colloc, 1))
            Z_coloc = np.random.uniform(self.Pos_min[2], self.Pos_max[2], (n_colloc, 1))
            self.collocation_data_np = np.array(np.meshgrid(X_coloc, Y_coloc,Z_coloc)).T.reshape(-1, 3)           # Combine the two arrays
            

        
        elif type=="random_2":
            #este tipo de random es una distribucion por todo el espacio para cada punto
            points=[]
            for _ in range(n_colloc):
                X_coloc_aux = np.random.uniform(self.Pos_min[0], self.Pos_max[0], 1)
                Y_coloc_aux = np.random.uniform(self.Pos_min[1], self.Pos_max[1], 1)
                Z_coloc_aux = np.random.uniform(self.Pos_min[2], self.Pos_max[2], 1)
                points.append(np.array([X_coloc_aux,Y_coloc_aux,Z_coloc_aux]))

            self.collocation_data_np = np.array(points).reshape(-1, 3)


In [5]:
data=Data("LINEAR_SMALL_DISP",10)
data.load_data()

In [6]:
data.create_colloc_points(10000)

In [7]:
px.scatter_3d(data_frame=pd.DataFrame(data.collocation_data_np,columns=["X","Y","Z"]),x="X",y="Y",z="Z")

# Redefinición de la PINN  

La pinn implementa el calculo de los gradientes.    

**Esta pinn solo da como resultado los desplazamientos, no las tensiones**

In [8]:
def normalize(X_tensor, X_min, X_max):
    """
    Normalize a tensor by columns given a max and min of each column.
    
    Parameters:
    X_tensor(numpy array): Tensor to be normalized.
    """
    # Normalize input vector X by each feature
    (num_row_X, num_col_X) = X_tensor.shape
    normalized_X_tensor = torch.zeros_like(X_tensor)
    
    # Fill the copy vector
    for col in range(num_col_X):
        min_X_col = X_min[col]
        max_X_col = X_max[col]
        normalized_X_tensor[:,col] = (X_tensor[:,col]-min_X_col)/(max_X_col-min_X_col)#(X_tensor[:,col]-min_X_col)*2/(max_X_col-min_X_col)# -1 #para que es este -1?  
        
    return normalized_X_tensor

def unnormalize(X_tensor, X_min, X_max):
    """
    Unnormalize a tensor by columns given a max and min of each column.
    
    Parameters:
    X_tensor(numpy array): Tensor to be unnormalized.
    X_max(numpy array): Max of each column of the tensor to be unnormalized.
    X_min(numpy array): Min of each column of the tensor to be unnormalized.
    """
    # Unnormalize input vector X by each feature
    (num_row_X, num_col_X) = X_tensor.shape
    unnormalized_X_tensor = torch.zeros_like(X_tensor)
    
    # Fill the copy vector
    for col in range(num_col_X):
        unnormalized_X_tensor[:,col] = (X_tensor[:,col])*(X_max[col]-X_min[col]) + X_min[col]#(X_tensor[:,col]+1)*(X_max[col]-X_min[col])/2 + X_min[col]
        
    return unnormalized_X_tensor

In [9]:
# Red neuronal  

class DNN(nn.Module):
    def __init__(self,layers,init_w="xavier"):
        super().__init__() 
        self.layers=layers
        self.activation = nn.Tanh()
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers) - 1)])
    
        if init_w=="xavier":
            #Xavier Normal Initialization
            for i in range(len(layers)-1):
                nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
                
                # set biases to zero
                nn.init.zeros_(self.linears[i].bias.data)
        # elif init_w=="gorot":
        #         nn.init. (self.linears[i].weight.data, gain=1.0)
                
        #         # set biases to zero
        #         nn.init.zeros_(self.linears[i].bias.data)


    def forward(self,x):
              
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)             

        #xn = normalize(x, Pos_min, Pos_max)

        # convert to float
        #a = x.float()

        # inpunt and hidden layers forward computation
        for i in range(len(self.layers)-2):
            z = self.linears[i](x)          
            x = self.activation(z)

        # output layer forward computation            
        a = self.linears[-1](x)
        
        return a

In [45]:
class PINN(DNN):
    '''
        Esta clase hace el registro de todos los parámetros, los cálculos de derivadas y de losses
        Y aplica las restricciones que hagan falta
    '''
    def __init__(self, layers,init_values=None,device=None):
        super().__init__(layers)
        if device is None:
            self.device= "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device=device
        self.to(self.device)

        #Define loss function
        self.loss_function = nn.MSELoss(reduction ='mean')

        #History of losses
        self.loss_history = {"Data": [],
                             "PDE": [],
                             "BC": [],
                             "Total":[]}
         #Parameters trials
        self.params_history = { "E": [] ,
                                "alpha": []}

        #inicialización de parametros
        self.nu = init_values[0]
        self.alpha = nn.Parameter(torch.tensor(init_values[2],dtype=torch.float32).to(self.device))
        self.E_ref = init_values[1]
        self.E = torch.tensor((1+self.alpha)*self.E_ref,dtype=torch.float32).to(self.device)

    def compute_XYZ(self, positions):
        # clone the input data and add AD
        pos = positions.clone().to(self.device)
        X = pos[:,0].reshape(-1,1)
        Y = pos[:,1].reshape(-1,1)
        Z = pos[:,2].reshape(-1,1)
        return X, Y, Z

    def compute_displacements(self,X, Y, Z):
        XYZ = torch.cat((X,Y,Z), dim=1).to( self.device )
        # Compute the output of the DNN
        U = self(XYZ)
        # Separating vector of directional displacements
        u = U[:,0].reshape(-1,1)
        v = U[:,1].reshape(-1,1)
        w = U[:,2].reshape(-1,1)
        return u, v, w

    # Los cálculos de las funciones de pérdida de hacen 1 vez, y luego ya vemos que hacemos
    # con esos datos, si usar pesos, si usar otras cosas...

    def loss_PDE(self, collocation_points, save = True):
        # estos son los puntos en los que imponemos las leyes físicas  
        # puede ser la ecuación de equilibrio, pero también pueden ser otras relaciones
        # que conozcamos entre inputs y outputs, como por ejemplo, relaciones termodinámicas etc..
        # si el output fuera posicion y tensión, entonces podriamos imponer la ley de hooke sobre este
        # input.
        X, Y, Z = self.compute_XYZ(collocation_points)
        u, v, w = self.compute_displacements(X,Y,Z)


        # y ahora tenemos que obtener las derivadas necesarias para aplicar la eq de equilibrio
        epsilon = self.compute_strain(X,Y,Z,u, v, w)
        sigma = self.compute_stress(epsilon)
        div_sigma = self.divergence(sigma,X, Y, Z)

        aux_value_loss_PDE=self.loss_function(div_sigma,torch.zeros_like(div_sigma).to(self.device))
        if save:
            self.loss_history["PDE"].append(aux_value_loss_PDE.to('cpu').detach().numpy())

    def loss_BC(self):

    def compute_gradU(self, X, Y, Z, U, V, W):

        # Compute the gradient of U
        Ux,Uy,Uz = autograd.grad(U, (X,Y,Z), torch.ones([X.shape[0], 1]).cuda(),retain_graph=True, create_graph=True)
        # Uy = autograd.grad(U, Y, torch.ones([Y.shape[0], 1]).cuda(),retain_graph=True, create_graph=True)[0]
        # Uz = autograd.grad(U, Z, torch.ones([Z.shape[0], 1]).cuda(),retain_graph=True, create_graph=True)[0]

        # Compute the gradient of V
        Vx,Vy,Vz = autograd.grad(V, (X,Y,Z), torch.ones([X.shape[0], 1]).cuda(),retain_graph=True, create_graph=True)
        # Vy = autograd.grad(V, Y, torch.ones([Y.shape[0], 1]).cuda(),retain_graph=True, create_graph=True)[0]
        # Vz = autograd.grad(V, Z, torch.ones([Z.shape[0], 1]).cuda(),retain_graph=True, create_graph=True)[0]

        # Compute the gradient of W
        Wx,Wy,Wz = autograd.grad(W, (X,Y,Z), torch.ones([X.shape[0], 1]).cuda(),retain_graph=True, create_graph=True)
        # Wy = autograd.grad(W, Y, torch.ones([Y.shape[0], 1]).cuda(),retain_graph=True, create_graph=True)[0]
        # Wz = autograd.grad(W, Z, torch.ones([Z.shape[0], 1]).cuda(),retain_graph=True, create_graph=True)[0]

        grad_u = torch.cat((Ux , Uy, Uz), dim=1).to(torch.float32)
        grad_v = torch.cat((Vx , Vy, Vz), dim=1).to(torch.float32)
        grad_w = torch.cat((Wx , Wy, Wz), dim=1).to(torch.float32)

        gradU = torch.cat((grad_u, grad_v, grad_w), dim=1).to(torch.float32).reshape(-1,3,3)

        return gradU

    def compute_strain(self, X,Y,Z,u, v, w):
        # Compute strain components using autograd
        nabla_U = self.compute_gradU(X,Y,Z,u, v, w).squeeze()
        strain = 0.5 * (nabla_U + nabla_U.swapaxes(1,2))
        return strain


    def compute_stress(self,strain):
        strain_flat=strain[:,(0,1,2,1,0,0),(0,1,2,2,2,1)]*torch.tensor([1,1,1,2,2,2],dtype=torch.float32).to(self.device)
        #strain_flat=strain[:,(0,1,2,0,0,1),(0,1,2,1,2,2)] # e00,e11,e22,e01,e02,e12
        C=(self.E/((1+self.nu)*(1-2*self.nu)))*torch.tensor(
                [[1-self.nu,self.nu,self.nu,0,0,0],
                [self.nu,1-self.nu,self.nu,0,0,0],
                [self.nu,self.nu,1-self.nu,0,0,0],
                [0,0,0,(1-2*self.nu)/2,0,0],
                [0,0,0,0,(1-2*self.nu)/2,0],
                [0,0,0,0,0,(1-2*self.nu)/2]]).float().to(self.device)

        return torch.matmul(C,strain_flat.T.float()).T.squeeze() #s11,s22,s33,s23,s13,s12

    def divergence(self,sigma, X, Y, Z):
        div_T = torch.zeros(sigma.shape[0],3)
        div_T[:,0]=autograd.grad(sigma[:,0],X,grad_outputs=torch.ones_like(sigma[:,0]),retain_graph=True)[0].squeeze()+autograd.grad(sigma[:,5],Y,grad_outputs=torch.ones_like(sigma[:,5]),retain_graph=True)[0].squeeze()+autograd.grad(sigma[:,4],Z,grad_outputs=torch.ones_like(sigma[:,4]),retain_graph=True)[0].squeeze()
        div_T[:,1]=autograd.grad(sigma[:,5],X,grad_outputs=torch.ones_like(sigma[:,5]),retain_graph=True)[0].squeeze()+autograd.grad(sigma[:,1],Y,grad_outputs=torch.ones_like(sigma[:,1]),retain_graph=True)[0].squeeze()+autograd.grad(sigma[:,3],Z,grad_outputs=torch.ones_like(sigma[:,3]),retain_graph=True)[0].squeeze()
        div_T[:,2]=autograd.grad(sigma[:,4],X,grad_outputs=torch.ones_like(sigma[:,4]),retain_graph=True)[0].squeeze()+autograd.grad(sigma[:,3],Y,grad_outputs=torch.ones_like(sigma[:,3]),retain_graph=True)[0].squeeze()+autograd.grad(sigma[:,2],Z,grad_outputs=torch.ones_like(sigma[:,2]),retain_graph=True)[0].squeeze()

        return div_T


# Entrenamiento

# DDEX y pruebas autograd

In [34]:

pinn=PINN([3,14,14,3],[2,3,4])


/tmp/ipykernel_10243/2404198933.py:30: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [35]:
# x1=torch.randn([30],requires_grad=True).to("cpu")
# x2=torch.randn([30],requires_grad=True).to("cpu")
# x3=torch.randn([30],requires_grad=True).to("cpu")
# X=torch.stack([x1,x2,x3],dim=1)
X=torch.randn([30,3],requires_grad=True).to("cuda")
x1=X[:,0].reshape(-1,1)
X.shape

torch.Size([30, 3])

In [36]:
y=pinn(X)
y

tensor([[ 0.2036,  0.2741, -0.3456],
        [ 0.0925,  0.3825,  0.2322],
        [-0.0454,  0.2009, -0.2854],
        [-0.3035, -0.3187, -0.2435],
        [-0.2676,  0.1590, -0.5100],
        [ 0.5444, -0.1023,  0.1319],
        [ 0.4544,  1.0185, -0.1494],
        [ 0.1568,  0.1912,  0.3833],
        [ 0.1458,  1.0734, -0.1743],
        [-0.3302, -1.2895,  0.2671],
        [-0.3320, -0.2111,  0.3653],
        [ 0.2395,  0.0842,  0.1687],
        [-0.0362, -0.8423,  0.5546],
        [ 0.0472, -0.3113, -0.2631],
        [-0.0980,  0.6193, -0.5986],
        [-0.4486, -0.3953,  0.2426],
        [ 0.0817,  0.7773, -0.5281],
        [-0.4070,  0.3180,  0.4533],
        [ 0.1936, -0.0874,  0.3108],
        [ 0.3120,  0.4554,  0.2953],
        [-0.4101,  0.6527, -0.1614],
        [-0.1685, -0.7659,  0.3039],
        [-0.4709, -0.7158, -0.2183],
        [-0.1363, -1.1246,  0.2580],
        [ 0.0629,  0.0626, -0.3761],
        [ 0.4773, -0.3780, -0.0095],
        [-0.3237,  0.3202, -0.4975],
 

In [37]:
x,y,z=pinn.compute_XYZ(X)

In [38]:
u,v,w=pinn.compute_displacements(x,y,z)

In [39]:
epsilon=pinn.compute_strain(x,y,z,u, v, w)

In [40]:
#strain_flat=strain[:,(0,1,2,1,0,0),(0,1,2,2,2,1)]*torch.tensor([1,1,1,2,2,2],dtype=torch.float32).to(device)
strain_flat=epsilon[:,(0,1,2,0,0,1),(0,1,2,1,2,2)]
strain_flat.shape

torch.Size([30, 6])

In [41]:
nu=2.0
E=1.0
C=(E/((1+nu)*(1-2*nu)))*torch.tensor(
        [[1-nu,nu,nu,0,0,0],
        [nu,1-nu,nu,0,0,0],
        [nu,nu,1-nu,0,0,0],
        [0,0,0,(1-2*nu)/2,0,0],
        [0,0,0,0,(1-2*nu)/2,0],
        [0,0,0,0,0,(1-2*nu)/2]]).float().to("cuda")
a=torch.matmul(C,strain_flat.T.float()).T.squeeze()

In [42]:
sigma=pinn.compute_stress(epsilon)
sigma.shape

torch.Size([30, 6])

In [43]:
pinn.divergence(sigma,x,y,z)

tensor([[ 0.2164, -0.7212,  0.5777],
        [-0.1004, -0.3578, -0.3169],
        [ 0.1412, -0.4319,  0.6826],
        [ 0.2173,  0.6775,  0.2942],
        [ 0.6122, -0.3196,  0.7825],
        [-0.7018,  0.3510,  0.0092],
        [-0.3423, -0.5681,  0.4467],
        [-0.6779,  0.5815, -0.6340],
        [ 0.1369, -1.5308,  0.7013],
        [ 0.4460,  0.4016, -0.1963],
        [-0.3432,  0.4088, -0.6983],
        [-0.2649, -0.2344, -0.2761],
        [-0.0352,  0.2835, -0.4109],
        [ 0.2494, -0.0865,  0.5268],
        [ 0.2281, -0.2082,  0.5426],
        [ 0.1375,  0.1141, -0.6114],
        [-0.0173, -0.3727,  0.3007],
        [-0.5411,  0.0882, -0.7284],
        [-0.3171,  0.1401, -0.6773],
        [-0.3241, -0.6587, -0.3398],
        [ 0.9779, -0.9985,  0.2715],
        [ 0.2255,  0.8108, -0.5545],
        [-0.1071,  1.2819, -0.0567],
        [ 0.0934,  1.1609, -0.6818],
        [ 0.4827, -0.8142,  0.6434],
        [-1.0092,  0.7332,  0.1085],
        [ 0.3417, -0.1929,  0.7881],
 

In [44]:
autograd.grad(sigma[:,0],(x,y),grad_outputs=torch.ones_like(sigma[:,0]),retain_graph=True)[1]


tensor([[ 0.6544],
        [-0.0213],
        [ 0.6410],
        [ 0.0293],
        [ 0.5704],
        [-0.3237],
        [ 0.7192],
        [-0.5146],
        [ 0.9117],
        [-0.3908],
        [-0.5862],
        [-0.1723],
        [-0.2496],
        [ 0.2317],
        [ 0.2310],
        [-0.6450],
        [ 0.2259],
        [-0.4093],
        [-0.5512],
        [-0.0313],
        [ 0.7927],
        [-0.7740],
        [-0.4406],
        [-0.7399],
        [ 0.6091],
        [-0.4515],
        [ 0.5983],
        [-0.3121],
        [ 0.1155],
        [ 0.7333]])

In [75]:
dde.gradients.jacobian(y,X,i=0,j=2)

tensor([[-0.1307],
        [-0.3491],
        [-0.3558],
        [-0.3587],
        [-0.3415],
        [-0.3003],
        [-0.3599],
        [-0.2743],
        [-0.3231],
        [-0.3407],
        [-0.2494],
        [-0.2730],
        [-0.2762],
        [-0.2505],
        [-0.3960],
        [-0.0742],
        [-0.2699],
        [-0.1811],
        [-0.2346],
        [-0.3323],
        [-0.2476],
        [-0.2374],
        [-0.1814],
        [-0.3360],
        [-0.2278],
        [-0.2573],
        [-0.3492],
        [-0.1643],
        [-0.3686],
        [-0.3068]], grad_fn=<SliceBackward0>)

In [60]:
u=y[:,0].reshape(-1,1)
v=y[:,1].reshape(-1,1)
w=y[:,2].reshape(-1,1)


In [229]:
autograd.grad(y,x1,grad_outputs=torch.ones(y.shape[0],3).to("cuda"),allow_unused=True,retain_graph=True)


(tensor([-0.0659,  0.2257, -0.1269, -0.1927,  0.2281,  0.0342,  0.1136,  0.0081,
          0.0216,  0.2476,  0.0974, -0.0360,  0.0762,  0.1903,  0.0621,  0.0816,
          0.0100, -0.0140,  0.2266,  0.0691, -0.0033,  0.1553,  0.1227, -0.1195,
          0.1057,  0.1504, -0.0533,  0.0926, -0.0106,  0.0489], device='cuda:0'),)

In [233]:
torch.sum(torch.stack([autograd.grad(u,x1,grad_outputs=torch.ones(u.shape[0],1).to("cuda"),allow_unused=True,retain_graph=True)[0],
                       autograd.grad(v,x1,grad_outputs=torch.ones(v.shape[0],1).to("cuda"),allow_unused=True,retain_graph=True)[0],
                       autograd.grad(w,x1,grad_outputs=torch.ones(w.shape[0],1).to("cuda"),allow_unused=True,retain_graph=True)[0]]),axis=0)

tensor([-0.0659,  0.2257, -0.1269, -0.1927,  0.2281,  0.0342,  0.1136,  0.0081,
         0.0216,  0.2476,  0.0974, -0.0360,  0.0762,  0.1903,  0.0621,  0.0816,
         0.0100, -0.0140,  0.2266,  0.0691, -0.0033,  0.1553,  0.1227, -0.1195,
         0.1057,  0.1504, -0.0533,  0.0926, -0.0106,  0.0489], device='cuda:0')

In [166]:
autograd.grad(c,b_aux,grad_outputs=torch.ones(3,1,3),allow_unused=True)

(None,)

In [108]:
c=torch.stack([a*2*b,b*2,a*b]).reshape(1,3,-1)
b=b.reshape(1,-1)


In [113]:
divergence(b,c)

RuntimeError: Mismatch in shape: grad_output[0] has a shape of torch.Size([1, 1]) and output[0] has a shape of torch.Size([3]).